# Porting DESeq into python using rpy2#

I will use a small example of [ERCC transcript](https://www.thermofisher.com/order/catalog/product/4456740) from [samples A and B in MAQC data](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3272078/).

In [72]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We will read the table and it should only contains count data of ERCC spikeins (rows) and 3 replicates from each of samples A and B (columns).

In [73]:
df = pd.read_table('../test/data/ercc1.txt')
df.head()

,TargetName,disease3_scan | 001 | PanCK,disease3_scan | 001 | neg,disease3_scan | 002 | PanCK,disease3_scan | 002 | neg,disease3_scan | 003 | PanCK,disease3_scan | 003 | neg,disease3_scan | 004 | PanCK,disease3_scan | 004 | neg,disease3_scan | 005 | PanCK,...,disease1B_scan | 015 | Geometric Segment,disease1B_scan | 016 | Geometric Segment,disease1B_scan | 017 | Geometric Segment,disease1B_scan | 018 | Geometric Segment,disease1B_scan | 019 | Geometric Segment,disease1B_scan | 020 | Geometric Segment,disease1B_scan | 021 | Geometric Segment,disease1B_scan | 022 | Geometric Segment,disease1B_scan | 023 | Geometric Segment,disease1B_scan | 024 | Geometric Segment
0,PADI2,15.0,35.0,36.0,41.0,9.0,37.0,42.0,37.0,14.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
1,CYP24A1,23.0,32.0,23.0,41.0,3.0,30.0,34.0,32.0,17.0,...,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1.0
2,SUPT16H,35.0,66.0,46.0,61.0,20.0,39.0,79.0,66.0,43.0,...,1.0,3.0,2.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0
3,ZMIZ2,51.0,81.0,87.0,113.0,68.0,84.0,164.0,118.0,81.0,...,3.0,2.0,2.0,1.0,1.0,3.0,2.0,2.0,3.0,2.0
4,SPAG9,48.0,65.0,46.0,91.0,25.0,58.0,95.0,81.0,45.0,...,8.0,10.0,7.0,3.0,1.0,7.0,5.0,4.0,6.0,2.0


In [74]:
# df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)
df.loc[:, df.columns != 'TargetName'] = df.loc[:, df.columns != 'TargetName'].astype(int)
df.head()

,TargetName,disease3_scan | 001 | PanCK,disease3_scan | 001 | neg,disease3_scan | 002 | PanCK,disease3_scan | 002 | neg,disease3_scan | 003 | PanCK,disease3_scan | 003 | neg,disease3_scan | 004 | PanCK,disease3_scan | 004 | neg,disease3_scan | 005 | PanCK,...,disease1B_scan | 015 | Geometric Segment,disease1B_scan | 016 | Geometric Segment,disease1B_scan | 017 | Geometric Segment,disease1B_scan | 018 | Geometric Segment,disease1B_scan | 019 | Geometric Segment,disease1B_scan | 020 | Geometric Segment,disease1B_scan | 021 | Geometric Segment,disease1B_scan | 022 | Geometric Segment,disease1B_scan | 023 | Geometric Segment,disease1B_scan | 024 | Geometric Segment
0,PADI2,15,35,36,41,9,37,42,37,14,...,1,1,3,1,1,1,1,2,1,1
1,CYP24A1,23,32,23,41,3,30,34,32,17,...,1,1,1,1,1,1,3,1,3,1
2,SUPT16H,35,66,46,61,20,39,79,66,43,...,1,3,2,1,1,3,1,2,1,1
3,ZMIZ2,51,81,87,113,68,84,164,118,81,...,3,2,2,1,1,3,2,2,3,2
4,SPAG9,48,65,46,91,25,58,95,81,45,...,8,10,7,3,1,7,5,4,6,2


And here, we will create a design matrix based on the samples in the count table. Note that the sample name has to be used as the ```pd.DataFrame``` index

In [83]:
sample_df = pd.read_table('../test/data/ercc2.txt')
sample_df.head()

,SlideName,ScanName,ROILabel,SegmentLabel,SegmentDisplayName,Sample_ID,AOISurfaceArea,AOINucleiCount,ROICoordinateX,ROICoordinateY,...,SequencingSaturation,UMIQ30,RTSQ30,disease_status,pathology,region,LOQ,NormalizationFactor,RoiReportX,RoiReportY
0,disease3,disease3_scan,7,Geometric Segment,disease3_scan | 007 | Geometric Segment,DSP-1001250007851-H-A02,31797.59253,202,11444,-18819,...,90.586744,0.9939,0.9932,DKD,abnormal,glomerulus,24.246524,0.556033,1197,-3538
1,disease3,disease3_scan,8,Geometric Segment,disease3_scan | 008 | Geometric Segment,DSP-1001250007851-H-A03,16920.10267,102,13064,-20021,...,90.707085,0.9948,0.9942,DKD,abnormal,glomerulus,19.338643,0.820810,1606,-3834
2,disease3,disease3_scan,9,Geometric Segment,disease3_scan | 009 | Geometric Segment,DSP-1001250007851-H-A04,14312.32987,98,14221,-18591,...,87.861873,0.9949,0.9943,DKD,abnormal,glomerulus,17.017533,0.957612,1892,-3474
3,disease3,disease3_scan,10,Geometric Segment,disease3_scan | 010 | Geometric Segment,DSP-1001250007851-H-A05,20032.83995,143,12120,-17683,...,89.399676,0.9925,0.9909,DKD,abnormal,glomerulus,22.335012,0.718209,1366,-3245
4,disease3,disease3_scan,11,Geometric Segment,disease3_scan | 011 | Geometric Segment,DSP-1001250007851-H-A06,27583.26127,195,12816,-16378,...,89.958230,0.9934,0.9923,DKD,abnormal,glomerulus,24.703838,0.492486,1542,-2925


In [84]:
sample_df.index = sample_df.SegmentDisplayName
sample_df.head()

,SlideName,ScanName,ROILabel,SegmentLabel,SegmentDisplayName,Sample_ID,AOISurfaceArea,AOINucleiCount,ROICoordinateX,ROICoordinateY,...,SequencingSaturation,UMIQ30,RTSQ30,disease_status,pathology,region,LOQ,NormalizationFactor,RoiReportX,RoiReportY
SegmentDisplayName,,,,,,,,,,,,,,,,,,,,,
disease3_scan | 007 | Geometric Segment,disease3,disease3_scan,7,Geometric Segment,disease3_scan | 007 | Geometric Segment,DSP-1001250007851-H-A02,31797.59253,202,11444,-18819,...,90.586744,0.9939,0.9932,DKD,abnormal,glomerulus,24.246524,0.556033,1197,-3538
disease3_scan | 008 | Geometric Segment,disease3,disease3_scan,8,Geometric Segment,disease3_scan | 008 | Geometric Segment,DSP-1001250007851-H-A03,16920.10267,102,13064,-20021,...,90.707085,0.9948,0.9942,DKD,abnormal,glomerulus,19.338643,0.820810,1606,-3834
disease3_scan | 009 | Geometric Segment,disease3,disease3_scan,9,Geometric Segment,disease3_scan | 009 | Geometric Segment,DSP-1001250007851-H-A04,14312.32987,98,14221,-18591,...,87.861873,0.9949,0.9943,DKD,abnormal,glomerulus,17.017533,0.957612,1892,-3474
disease3_scan | 010 | Geometric Segment,disease3,disease3_scan,10,Geometric Segment,disease3_scan | 010 | Geometric Segment,DSP-1001250007851-H-A05,20032.83995,143,12120,-17683,...,89.399676,0.9925,0.9909,DKD,abnormal,glomerulus,22.335012,0.718209,1366,-3245
disease3_scan | 011 | Geometric Segment,disease3,disease3_scan,11,Geometric Segment,disease3_scan | 011 | Geometric Segment,DSP-1001250007851-H-A06,27583.26127,195,12816,-16378,...,89.958230,0.9934,0.9923,DKD,abnormal,glomerulus,24.703838,0.492486,1542,-2925


In [119]:
l2 = set(sample_df.SegmentDisplayName.unique())
l1 = set(df.columns[1:].tolist())
print(len(l1), len(l2))

231 231


In [120]:
# using == to check if 
# lists are equal
if l1 == l2:
    print ("The lists are identical")
else :
    print ("The lists are not identical")

The lists are identical


Running DESeq2 is jsut like how it is run in ```R```, but instead of the row.name being gene ID for the count table, we can jsut tell the function which column is the gene ID:

In [122]:
from diffexpr.py_deseq import py_DESeq2

dds = py_DESeq2(count_matrix = df,
               design_matrix = sample_df,
               design_formula = '~ disease_status',
               gene_column = 'TargetName') # <- telling DESeq2 this should be the gene ID column
    
dds.run_deseq() 
dds.get_deseq_result(contrast = ['SlideName','disease3','normal3'])
res = dds.deseq_result 
res.head()

  rownames of the colData:
   disease3_scan | 007 | Geometric Segment,disease3_scan | 008 | Geometric Segment,disease3_scan | 009 | Geometric Segment,disease3_scan | 010 | Geometric Segment,disease3_scan | 011 | Geometric Segment,disease3_scan | 012 | Geometric Segment,disease3_scan | 013 | Geometric Segment,disease3_scan | 014 | Geometric Segment,disease3_scan | 015 | Geometric Segment,disease3_scan | 016 | Geometric Segment,disease3_scan | 017 | Geometric Segment,disease3_scan | 018 | Geometric Segment,disease3_scan | 019 | Geometric Segment,disease3_scan | 020 | Geometric Segment,disease3_scan | 021 | Geometric Segment,disease3_scan | 022 | Geometric Segment,disease3_scan | 023 | Geometric Segment,disease3_scan | 024 | Geometric Segment,disease3_scan | 025 | Geometric Segment,disease3_scan | 026 | Geometric Segment,disease3_scan | 027 | Geometric Segment,disease3_scan | 028 | Geometric Segment,disease3_scan | 029 | Geometric Segment,disease3_scan | 030 | Geometric Segment,di



RRuntimeError: Error in (function (countData, colData, design, tidy = FALSE, ignoreRank = FALSE,  : 
  rownames of the colData:
   disease3_scan | 007 | Geometric Segment,disease3_scan | 008 | Geometric Segment,disease3_scan | 009 | Geometric Segment,disease3_scan | 010 | Geometric Segment,disease3_scan | 011 | Geometric Segment,disease3_scan | 012 | Geometric Segment,disease3_scan | 013 | Geometric Segment,disease3_scan | 014 | Geometric Segment,disease3_scan | 015 | Geometric Segment,disease3_scan | 016 | Geometric Segment,disease3_scan | 017 | Geometric Segment,disease3_scan | 018 | Geometric Segment,disease3_scan | 019 | Geometric Segment,disease3_scan | 020 | Geometric Segment,disease3_scan | 021 | Geometric Segment,disease3_scan | 022 | Geometric Segment,disease3_scan | 023 | Geometric Segment,disease3_scan | 024 | Geometric Segment,disease3_scan | 025 | Geometric Segment,disease3_scan | 026 | Geometric Segment,disease3_scan | 027 | Geometric Segment,disease3_scan | 028 | Geometr

In [ ]:
dds.normalized_count() #DESeq2 normalized count

In [ ]:
dds.comparison # show coefficients for GLM

In [ ]:
# from the last cell, we see the arrangement of coefficients, 
# so that we can now use "coef" for lfcShrink
# the comparison we want to focus on is 'sample_B_vs_A', so coef = 4 will be used
lfc_res = dds.lfcShrink(coef=4, method='apeglm')
lfc_res.head()